# Collective Agreement Virtual Assistant
LangChain, AzureChatOpenAI, OpenAIEmbedding

In [31]:
# Install software packages:
%pip install langchain-openai==0.2.0
%pip install python-dotenv==1.0.0
%pip install langchain==0.3.4
%pip install langchainhub==0.1.13
%pip install pypdf==3.17.0
%pip install tiktoken==0.5.1
%pip install chromadb
%pip install langchain-community==0.3.20
%pip install unstructured
%pip install python-docx
%pip install unstructured langchain-community


Note: you may need to restart the kernel to use updated packages.
  Using cached langchain-0.3.4-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.4-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.20 requires langchain<1.0.0,>=0.3.21, but you have langchain 0.3.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
  Using cached pypdf-3.17.0-py3-none-any.whl.metadata (7.5 kB)
Using cached pypdf-3.17.0-py3-none-any.whl (277 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.5.0
    Uninstalling pypdf-5.5.0:
      Successfully uninstalled pypdf-5.5.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.34.0 requires pypdf>=4.0, but you have pypdf 3.17.0 which is incompatible.


  Using cached tiktoken-0.5.1.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build tiktoken
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      C:\Users\gscgo\AppData\Local\Temp\pip-build-env-ufjtrn_0\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ************************************************

  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-4.0.1-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached onnxruntime-1.22.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached opentelemetry_api-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.33.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_sdk-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached kubernetes-32.0.1-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached google_auth-2.40.1-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any

In [32]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

os.environ["ALLOW_RESET"] = "TRUE"

load_dotenv('.env', override=True)

# # Explicitly set the deployment name to match your Azure deployment
embedding_model = AzureOpenAIEmbeddings(
    model=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT'),
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDING_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_EMBEDDING_API_VERSION'),
    api_key=os.getenv('AZURE_OPENAI_EMBEDDING_API_KEY')
    )

# Instantiate an AzureOpenAI LLM:
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [33]:
#load web page from MOM PWM URLS:
from langchain.document_loaders import WebBaseLoader
loader_multiple_pages = WebBaseLoader(["https://www.mom.gov.sg/employment-practices/progressive-wage-model/cleaning-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/retail-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/food-services-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/occupational-pws-for-administrators-and-drivers",
                                      "https://www.mom.gov.sg/passes-and-permits/work-permit-for-foreign-worker/sector-specific-rules/non-traditional-sources-occupation-list"
                                      ])

## List of all pdf files (Tripartite Guidelines and)
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("Tripartite Guidelines")

#load CAs in docx format
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader

folder_path="wst"
txt_loader = DirectoryLoader(folder_path, glob="*.docx", loader_cls=UnstructuredWordDocumentLoader)
txt_documents = txt_loader.load()

In [35]:
# Split content into chunks.
# Import a splitter module called RecursiveCharacterTextSplitter:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Instantiate the splitter:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Do the splitting:
splits = text_splitter.split_documents(loader.load())

# Embed and store splits in Chroma database.
# Import the Chroma and embeddings modules:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# Instantiate the Chroma vector store with arguments:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model
)
# Instantiate a retriever based on the vector store:
retriever = vectorstore.as_retriever() 

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}